In [ ]:
### import torch.nn as nn
from torchvision import models
F = open('ECG_color1C_VGG16.txt', 'w')  
# F = open('test', 'w') 
whole=[True]
method=["gasf","gadf","mtf"]
# method=[["gasf","gadf","mtf"]]
# method=[["gasf","gadf"],["mtf","gasf"],["mtf","gadf"]]
for l in method:
    for m in whole:
        for k in range(20):
            accuracy_matrix =[]
            start_time = time.time()
            for i in range(5):
                min_error=100
                min_lose=100
                count=0
                number=1
#                 cross = dataset_six_combine_method.dataset_dataloader(True,True,"mtf","gasf",i+1)
                cross = dataset_color.dataset_dataloader("color_",True,True,l,i+1,batch_size=20)
#                 cross = dataset_color.dataset_dataloader("color_",True,True,l,i+1,batch_size=20)
#                 cross = dataset_color.dataset_dataloader("64_color_",True,True,l,i+1)
#                 cross = dataset_color.dataset_dataloader("color_",True,True,l[0],l[0],l[0],i+1,batch_size=15)
#                 cross = dataset_color_combine.dataset_dataloader("color_",True,True,l[0],l[1],i+1,batch_size=20)
                EPOCH = 200              # train the training data n times, to save time, we just train 1 epoch
                LR = 0.00023
#                 LR = 3e-4

                vgg16 = models.vgg16_bn()
                vgg16.features[0] = nn.Conv2d(3,64,3,1,1)
                vgg16.classifier[0] = nn.Linear(512*4*8, 4096)
#                 vgg16.classifier[3] = nn.Linear(1024, 1024)
                vgg16.classifier[-1] = nn.Linear(4096,2)

                
                cnn=vgg16
                cnn.cuda()
            #     print(cnn)

                optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
                loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

                # following function (plot_with_labels) is for visualization, can be ignored if not interested

                # training and testing


                for epoch in range(EPOCH):
                    for step, (x, y) in enumerate(cross.train_loader):   # gives batch data, normalize x when iterate train_loader
                        b_x = Variable(x).cuda()   # batch x
                        b_y = Variable(y).cuda()   # batch y
                        cnn.train()
                        output = cnn(b_x)             # cnn output
                        loss = loss_func(output, b_y)   # cross entropy loss
                        del b_x, b_y, output
                        optimizer.zero_grad()           # clear gradients for this training step
                        loss.backward()                 # backpropagation, compute gradients
                        optimizer.step() # apply gradients
#                         del loss                       
                        torch.cuda.empty_cache()
                        
                    cnn.eval()
                    count_step = 0
                    acc_step = 0
                    with torch.no_grad():
                        for test_step, (test_x, test_y) in enumerate(cross.test_loader):                                                                
                            test_x = Variable(test_x).cuda()   # batch x
                            test_y = Variable(test_y).cuda()   # batch y
                            test_output= cnn(test_x)
                            pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()
                            acc_step += sum(pred_y == test_y).item()

#                             cnn.train()
                    accuracy = acc_step/float(cross.test_y.size(0))
#                         accuracy = sum(pred_y == cross.test_y).item() / float(cross.test_y.size(0))
                    error  = (1-accuracy)*100

                    if error<min_error:
                        min_error=error
                        min_lose=100
                        count=0
                    else:
                        count+=1
                        print(".",end="")

                    if error == min_error:
                        if loss.data.item()<min_lose:
                            min_lose=loss.data.item()
        #                     torch.save(cnn,'cnn_save/cnn'+str(number)+'.pkl')
                            number+=1
            #                 print('save!!')
            #                 print('\ncount: ',count,'Epoch: ', epoch, '| train loss: %.15f' % loss.data[0], '| min validation error rate: %.20f' % min_error)

                    if count>=15:
                        break
                print("min_error:",min_error)
                # print 10 predictions from test data
                # test_output, _ = cnn(test_x)
                # pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()

                # print(pred_y, 'prediction number')
                # print(test_y, 'real number')
                accuracy_matrix.append(min_error)
                print('\n number: ',number-1 ,min_error)

            
            print(accuracy_matrix)
            sum(accuracy_matrix) / float(len(accuracy_matrix))
#             F.write(str(l[0])+str(l[1])+str(l[2])+","+str(m)+","+str(k)+",")
#             F.write(str(l[0])+str(l[1])+","+str(m)+","+str(k)+",")
            F.write(str(l)+","+str(m)+","+str(k)+",")
            for i in accuracy_matrix:  
                F.write(str(i) + ",")
            F.write(str(sum(accuracy_matrix) / float(len(accuracy_matrix)))+",")
            F.write(str(time.time() - start_time)+"\n")
            print(time.time() - start_time)
F.close()  